In [8]:
import datasets
ds = datasets.load_dataset('deepmind/code_contests', split='train')

Resolving data files:   0%|          | 0/39 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/24 [00:00<?, ?it/s]

In [9]:
def map_code_contests(example):
    question = example['description']
    test_inputs = [x.strip().split('\n') for x in example['private_tests']['input']]
    test_outputs = [x.strip() for x in example['private_tests']['output']]
    max_time = float(example['time_limit']['seconds'])
    
    return {
        'question': question,
        'test_inputs': test_inputs,
        'test_outputs': test_outputs,
        'max_time': max_time
    }
ds = ds.select(range(100))
ds = ds.filter(lambda ex: ex['difficulty']  == 7 and '<image>' not in ex['description']) # filter easy samples 
ds = ds.map(map_code_contests, remove_columns=ds.column_names)
        

In [10]:
list(ds[0].keys())

['question', 'test_inputs', 'test_outputs', 'max_time']

In [ ]:
import io
import sys

def test_solution(test_case):
    test_input = test_case['test_inputs']
    test_output = test_case['test_outputs']
    # Define the solution code as a string
    solution = test_case['solution']
    res = []
    for i in range(len(test_input)):
        # Redirect stdout to capture print statements in the solution code
        output_capture = io.StringIO()
        original_stdout = sys.stdout
        sys.stdout = output_capture
        inp = iter(test_input[i])
        # Execute the solution code with a custom input function
        try:
            # Set up a local environment with a mock input function
            local_vars = {"input": lambda: next(inp)}

            # Execute the solution in this environment
            exec(solution, {}, local_vars)
        finally:
            # Restore the original stdout
            sys.stdout = original_stdout

        # Retrieve the captured output
        captured_output = output_capture.getvalue().strip()

        # Compare the captured output to the expected output
        if captured_output == test_output[i]:
            res.append(1)
        else:
            #print(test_case['solution'], end='------------------\n')

            res.append(0)
    return sum(res) / len(res)

for i,c in enumerate(modified_dataset):
    print(f"case number {i} has {test_solution(c)*100} % correct tests")

In [ ]:
print('\n'.join([f"{k}: {v}" for k,v in modified_dataset[0].items()]))